In [2]:
# 引入库
import dgl
import dgl.nn as dglnn
from dgl.dataloading.neighbor import MultiLayerNeighborSampler
from dgl.dataloading.pytorch import NodeDataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

from yaml import load, Loader
from utils import load_dgl_graph, set_seed_logger

In [ ]:
# 定义Encoder模型
class GraphAttnEncoder(nn.Module):

    def __init__(self,
                 in_feats,
                 hidden_dim,
                 n_layers,
                 heads,
                 activation,
                 feat_drop,
                 attn_drop
                 ):
        super(GraphAttnEncoder, self).__init__()
        self.in_feats = in_feats
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.heads = heads
        self.feat_dropout = feat_drop
        self.attn_dropout = attn_drop
        self.activation = activation

        self.layers = nn.ModuleList()

        # build multiple layers
        self.layers.append(dglnn.GATConv(in_feats=self.in_feats,
                                         out_feats=self.hidden_dim,
                                         num_heads=self.heads[0],
                                         feat_drop=self.feat_dropout,
                                         attn_drop=self.attn_dropout,
                                         activation=self.activation))

        for l in range(1, (self.n_layers)):
            # due to multi-head, the in_dim = num_hidden * num_heads
            self.layers.append(dglnn.GATConv(in_feats=self.hidden_dim * self.heads[l - 1],
                                             out_feats=self.hidden_dim,
                                             num_heads=self.heads[l],
                                             feat_drop=self.feat_dropout,
                                             attn_drop=self.attn_dropout,
                                             activation=self.activation))

    def forward(self, blocks, features):
        h = features

        for l in range(self.n_layers - 1):
            h = self.layers[l](blocks[l], h).flatten(1)
        return h

In [ ]:
# loss
class SimLoss(nn.Module):
    def __init__(self):
        super(SimLoss,self).__init__()

    def forward(self, out_1, out_2,temperature=0.5):
        # 分母 ：X.X.T，再去掉对角线值，分析结果一行，可以看成它与除了这行外的其他行都进行了点积运算（包括out_1和out_2）,
        # 而每一行为一个batch的一个取值，即一个输入图像的特征表示，
        # 因此，X.X.T，再去掉对角线值表示，每个输入图像的特征与其所有输出特征（包括out_1和out_2）的点积，用点积来衡量相似性
        # 加上exp操作，该操作实际计算了分母
        B, _ = out_1.shape 
        # [2*B, D]
        out = torch.cat([out_1, out_2], dim=0)
        # [2*B, 2*B]
        sim_matrix = torch.exp(torch.mm(out, out.t().contiguous()) / temperature)
        mask = (torch.ones_like(sim_matrix) - torch.eye(2 * B, device=sim_matrix.device)).bool()
        # [2*B, 2*B-1]
        sim_matrix = sim_matrix.masked_select(mask).view(2 * B, -1)

        # 分子： *为对应位置相乘，也是点积
        # compute loss
        pos_sim = torch.exp(torch.sum(out_1 * out_2, dim=-1) / temperature)
        # [2*B]
        pos_sim = torch.cat([pos_sim, pos_sim], dim=0)
        return (- torch.log(pos_sim / sim_matrix.sum(dim=-1))).mean()

In [ ]:
# # 定义dataloader
# class simDataset(Dataset):
#     def __init__(self, n_ids):
#         self.n_ids = n_ids
#     def __len__(self):
#         return len(self.n_ids)
#     def __getitem__(self, idx):
#         return self.n_ids[idx]
def graph_resample():
    pass

In [ ]:
def get_dataloader(dataset_cfg, graph_data):
    graph, labels, train_nid, val_nid, test_nid, node_feat = graph_data
    sampler = MultiLayerNeighborSampler(dataset_cfg['FANOUTS'])  # 50

    train_dataloader = NodeDataLoader(graph,
                                      train_nid,
                                      sampler,
                                      batch_size=dataset_cfg['BATCH_SIZE'],
                                      shuffle=True,
                                      drop_last=False,
                                      num_workers=dataset_cfg['NUM_WORKERS'])
    val_dataloader = NodeDataLoader(graph,
                                    val_nid,
                                    sampler,
                                    batch_size=dataset_cfg['BATCH_SIZE'],
                                    shuffle=True,
                                    drop_last=False,
                                    num_workers=dataset_cfg['NUM_WORKERS'])

    return train_dataloader, val_dataloader, node_feat, labels

In [3]:
def load_subtensor(node_feats, labels, seeds, input_nodes, n_classes, device, training=False):
    """
    Copys features and labels of a set of nodes onto GPU.
    """
    input_feats = node_feats[input_nodes].to(device)

    masklabels = labels.clone()
    masklabels[seeds] = -1
    input_labels = masklabels[input_nodes]

    if training:
        rd_m = torch.rand(input_labels.shape[0])
        rd_y = torch.randint(0, n_classes, size=input_labels.shape)
        input_labels[rd_m < 0.15] = -1
        input_labels[rd_m > 0.97] = rd_y[rd_m > 0.97]

    input_labels[input_labels < 0] = n_classes
    input_labels = input_labels.to(device)

    batch_labels = labels[seeds].to(device)
    return input_feats, input_labels, batch_labels

In [ ]:
def train_epoch(epoch, model, train_dataloader, node_feat, labels, optimizer, criterion, n_classes,
                device, global_step, log_step):
    # 获得数据对
            input_nodes, output_nodes, blocks = batch_data
            input_feats, input_labels, batch_labels = load_subtensor(node_feat,
                                                                 labels,
                                                                 output_nodes,
                                                                 input_nodes,
                                                                 dataset_cfg['N_CLASS'],
                                                                 device,
                                                                 training=False)
            blocks = [block.to(device) for block in blocks]

            _batch_data = train_dataloader.collator.collate(output_nodes)
            _input_nodes, _output_nodes, _blocks = _batch_data
            _input_feats, _input_labels, _batch_labels = load_subtensor(node_feat,
                                                        labels,
                                                        _output_nodes,
                                                        _input_nodes,
                                                        dataset_cfg['N_CLASS'],
                                                        device,
                                                        training=False)

            _blocks = [_block.to(device) for _block in _blocks]


            pre_L = model(blocks, input_feats, input_labels)
            pre_R = model(_blocks, _input_feats, _input_labels)

            loss=lossLR(pre_L,pre_R)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            print("epoch", epoch, "batch", batch, "loss:", loss.detach().item())
            total_loss += loss.detach().item()

In [ ]:
# train stage one
def train(model_cfg, dataset_cfg, train_cfg, device, graph_data):

    # 获得数据
    train_dataloader, val_dataloader, node_feat, labels = get_dataloader(dataset_cfg, graph_data)
    
    gat_cfg = model_cfg['GAT']

    model = GraphAttnEncoder(in_feats=gat_cfg['IN_FEATS'], 
                            hidden_dim=gat_cfg['HIDDEN_DIM'],
                            n_layers=gat_cfg['HIDDEN_DIM'],
                            activation=F.relu,
                            feat_drop=gat_cfg['GAT_CFG'],
                            attn_drop=gat_cfg['ATTN_DROP'])
                            
    lossLR=SimLoss()
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)

    for epoch in range(1, train_cfg['EPOCH']+1):
        model.train()
        total_loss = 0
        for batch, batch_data in enumerate(train_dataloader):
            

        print("epoch loss:",total_loss/len(train_dataset)*args.batch_size)

        with open(os.path.join(config.save_path, "stage1_loss.txt"), "a") as f:
            f.write(str(total_loss/len(train_dataset)*args.batch_size) + " ")

        if epoch % 5==0:
            torch.save(model.state_dict(), os.path.join(config.save_path, 'model_stage1_epoch' + str(epoch) + '.pth'))

In [10]:
config_path = 'config.yaml'
f = open(config_path, 'r', encoding='utf-8')
config = f.read()
config = load(config, Loader=Loader)
dataset_cfg = config['DATASET']
model_cfg = config['MODEL']
train_cfg = config['TRAIN']

In [11]:
# 加载图
k_fold = 2
graph_data = load_dgl_graph(dataset_cfg['DATA_PATH'],
                                k_fold,
                                norm_feature=dataset_cfg['NORM_FEATURE'])

################ Graph info: ###############
Graph(num_nodes=3655452, num_edges=61832630,
      ndata_schemes={}
      edata_schemes={})
demo:   (835533,)
(208884,)
################ Label info: ################
Total labels (including not labeled): 3655452
               Training label number: 835533
             Validation label number: 208884
                   Test label number: 592391
################ Feature info: ###############
Node's feature shape:torch.Size([3655452, 300])


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
train(model_cfg, dataset_cfg, train_cfg,  device, graph_data)

In [1]:
import torch
from torch.distributions.categorical import Categorical
probs = torch.FloatTensor([[0.05,0.1,0.85],[0.05,0.05,0.9]])
 
dist = Categorical(probs)
print(dist)
# Categorical(probs: torch.Size([2, 3]))
 
index = dist.sample()
print(index.numpy())
# [2 2]

Categorical(probs: torch.Size([2, 3]))
[2 2]


In [20]:
index = dist.sample()
print(index.numpy())

[1 2]
